In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover , HashingTF , Tokenizer
from pyspark.sql.functions import split
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression 
from pyspark.sql import functions as f
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
spark = SparkSession.builder.appName("Sentiment").getOrCreate()

In [3]:
data_1 = sc.textFile("amazon_cells_labelled.txt")
data_2 = sc.textFile("imdb_labelled.txt")
data_3 = sc.textFile("yelp_labelled.txt")
df_4  = spark.read.csv("tweets.csv",header=False,inferSchema=True)
df_4.cache()

DataFrame[_c0: int, _c1: bigint, _c2: string, _c3: string, _c4: string, _c5: string]

In [4]:
df_4 = df_4.withColumnRenamed("_c0","label")
df_4 = df_4.withColumnRenamed("_c1","ID")
df_4 = df_4.withColumnRenamed("_c2","Date")
df_4 = df_4.withColumnRenamed("_c3","Query")
df_4 = df_4.withColumnRenamed("_c4","username")
df_4 = df_4.withColumnRenamed("_c5","Tweets")

In [5]:
df_4.show(10)

+-----+----------+--------------------+--------+---------------+--------------------+
|label|        ID|                Date|   Query|       username|              Tweets|
+-----+----------+--------------------+--------+---------------+--------------------+
|    0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|    0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|    0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|    0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|    0|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|    0|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|    0|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|    0|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4

In [6]:
rec_data=df_4.select("label","Tweets")

In [7]:
positive = rec_data.select("*").filter(rec_data["label"]==4)
negative = rec_data.select("*").filter(rec_data["label"]==0)
#negative.repartition(1).write.csv("negative" , sep='|')
#positive.repartition(1).write.csv("positive.csv", sep='|')


In [8]:
positive = positive.withColumnRenamed("tweets","statement")
negative = negative.withColumnRenamed("tweets","statement")
positive = positive.select("statement" , f.when(positive["label"]==4 , 1).otherwise(0))
positive = positive.withColumnRenamed("CASE WHEN (label = 4) THEN 1 ELSE 0 END","label")
negative = negative.select("statement" , f.when(negative["label"]==0 , 0).otherwise(1))
negative = negative.withColumnRenamed("CASE WHEN (label = 0) THEN 0 ELSE 1 END","label")
positive.show()
negative.show()

+--------------------+-----+
|           statement|label|
+--------------------+-----+
|I LOVE @Health4Ua...|    1|
|im meeting up wit...|    1|
|@DaRealSunisaKim ...|    1|
|Being sick can be...|    1|
|@LovesBrooklyn2 h...|    1|
|@ProductOfFear Yo...|    1|
|@r_keith_hill Tha...|    1|
|@KeepinUpWKris I ...|    1|
|@tommcfly ah, con...|    1|
|@e4VoIP I RESPOND...|    1|
|crazy day of scho...|    1|
|@naughtyhaughty H...|    1|
|@nileyjileyluver ...|    1|
|@soundwav2010 At ...|    1|
|@LutheranLucciol ...|    1|
|Just added tweeti...|    1|
|@michellardi i re...|    1|
|@nicolerichie: yo...|    1|
|Catching Up on Em...|    1|
|Dancing around th...|    1|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|           statement|label|
+--------------------+-----+
|@switchfoot http:...|    0|
|is upset that he ...|    0|
|@Kenichan I dived...|    0|
|my whole body fee...|    0|
|@nationwideclass ...|    0|
|@Kwesidei not the...|    0|
|         Need a 

In [9]:
data_1 = data_1.map(lambda x:x.split("\t"))
data_2=data_2.map(lambda x:x.split("\t"))
data_3=data_3.map(lambda x:x.split("\t"))

In [10]:
data_1.take(1)
data_2.take(1)
data_3.take(1)

[['Wow... Loved this place.', '1']]

In [11]:
df_1 = data_1.toDF()
df_2 = data_2.toDF()
df_3 = data_3.toDF()

In [12]:
df_1.show()
df_2.show()
df_3.show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|So there is no wa...|  0|
|Good case, Excell...|  1|
|Great for the jaw...|  1|
|Tied to charger f...|  0|
|   The mic is great.|  1|
|I have to jiggle ...|  0|
|If you have sever...|  0|
|If you are Razr o...|  1|
|Needless to say, ...|  0|
|What a waste of m...|  0|
|And the sound qua...|  1|
|He was very impre...|  1|
|If the two were s...|  0|
|Very good quality...|  1|
|The design is ver...|  0|
|Highly recommend ...|  1|
|I advise EVERYONE...|  0|
|    So Far So Good!.|  1|
|       Works great!.|  1|
|It clicks into pl...|  0|
+--------------------+---+
only showing top 20 rows

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|A very, very, ver...|  0|
|Not sure who was ...|  0|
|Attempting artine...|  0|
|Very little music...|  0|
|The best scene in...|  1|
|The rest of the m...|  0|
| Wasted two hours.  |  0|
|Saw the movie tod...|  1|
|A bit predictable.  |  0|
|L

In [13]:
df = df_1.union(df_2)

In [14]:
df = df.union(df_3)

In [15]:
df = df.union(positive)

In [16]:
df = df.union(negative)

In [17]:
df.count()

1603000

In [18]:
df = df.withColumnRenamed("_1","statement")
df=df.withColumnRenamed("_2","label")
df = df.withColumn("label" , df["label"].cast(IntegerType()))

In [19]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[statement: string, label: int]>

In [26]:
df.show()

+--------------------+-----+
|           statement|label|
+--------------------+-----+
|So there is no wa...|    0|
|Good case, Excell...|    1|
|Great for the jaw...|    1|
|Tied to charger f...|    0|
|   The mic is great.|    1|
|I have to jiggle ...|    0|
|If you have sever...|    0|
|If you are Razr o...|    1|
|Needless to say, ...|    0|
|What a waste of m...|    0|
|And the sound qua...|    1|
|He was very impre...|    1|
|If the two were s...|    0|
|Very good quality...|    1|
|The design is ver...|    0|
|Highly recommend ...|    1|
|I advise EVERYONE...|    0|
|    So Far So Good!.|    1|
|       Works great!.|    1|
|It clicks into pl...|    0|
+--------------------+-----+
only showing top 20 rows



In [21]:
token = Tokenizer(inputCol = "statement",outputCol="words")
s_rem = StopWordsRemover(inputCol=token.getOutputCol(),outputCol="filtered")
tf = HashingTF(inputCol=s_rem.getOutputCol(),outputCol = "features")
rec_data = token.transform(df)
rec_data = s_rem.transform(rec_data)
rec_data = tf.transform(rec_data)

In [22]:
rec_data.show()

+--------------------+-----+--------------------+--------------------+--------------------+
|           statement|label|               words|            filtered|            features|
+--------------------+-----+--------------------+--------------------+--------------------+
|So there is no wa...|    0|[so, there, is, n...|[way, plug, us, u...|(262144,[21872,61...|
|Good case, Excell...|    1|[good, case,, exc...|[good, case,, exc...|(262144,[113432,1...|
|Great for the jaw...|    1|[great, for, the,...|   [great, jawbone.]|(262144,[138356,2...|
|Tied to charger f...|    0|[tied, to, charge...|[tied, charger, c...|(262144,[22836,35...|
|   The mic is great.|    1|[the, mic, is, gr...|       [mic, great.]|(262144,[52197,80...|
|I have to jiggle ...|    0|[i, have, to, jig...|[jiggle, plug, ge...|(262144,[99895,11...|
|If you have sever...|    0|[if, you, have, s...|[several, dozen, ...|(262144,[8443,829...|
|If you are Razr o...|    1|[if, you, are, ra...|[razr, owner...yo...|(262144,[8

In [23]:
val_data = rec_data.select("label","features")

In [24]:
train , test = val_data.randomSplit([0.8,0.2])


In [25]:
train.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|(262144,[14,13381...|
|    0|(262144,[14,24095...|
|    0|(262144,[14,58236...|
|    0|(262144,[925,3000...|
|    0|(262144,[1145,362...|
|    0|(262144,[1846,490...|
|    0|(262144,[2362,212...|
|    0|(262144,[2437,470...|
|    0|(262144,[2711,300...|
|    0|(262144,[2937,330...|
|    0|(262144,[3114,936...|
|    0|(262144,[3121,362...|
|    0|(262144,[4799,102...|
|    0|(262144,[5381,112...|
|    0|(262144,[5765,121...|
|    0|(262144,[5765,335...|
|    0|(262144,[6782,304...|
|    0|(262144,[7290,154...|
|    0|(262144,[8154,697...|
|    0|(262144,[8217,790...|
+-----+--------------------+
only showing top 20 rows



In [27]:
lr =LogisticRegression()
model = lr.fit(train)

In [28]:
pred = model.transform(test)

In [29]:
predi = pred.select("label","prediction" , f.when(pred["prediction"]==1.0 , "Positive").otherwise("Negative"))

In [30]:
predi = predi.withColumnRenamed("CASE WHEN (prediction = 1.0) THEN Positive ELSE Negative END" , "Predicted Review(positive/negative)")

In [31]:
predi.show()
pred.show()

+-----+----------+-----------------------------------+
|label|prediction|Predicted Review(positive/negative)|
+-----+----------+-----------------------------------+
|    0|       0.0|                           Negative|
|    0|       0.0|                           Negative|
|    0|       1.0|                           Positive|
|    0|       0.0|                           Negative|
|    0|       0.0|                           Negative|
|    0|       1.0|                           Positive|
|    0|       1.0|                           Positive|
|    0|       0.0|                           Negative|
|    0|       0.0|                           Negative|
|    0|       0.0|                           Negative|
|    0|       0.0|                           Negative|
|    0|       1.0|                           Positive|
|    0|       0.0|                           Negative|
|    0|       1.0|                           Positive|
|    0|       0.0|                           Negative|
|    0|   

In [32]:
evalu = MulticlassClassificationEvaluator(labelCol="label" , predictionCol="prediction",metricName="accuracy")
acc = evalu.evaluate(pred)

In [33]:
print(acc)

0.7171360584845424


In [58]:
review = [input("Please Enter the Review:")]

Please Enter the Review:i'l pry frm gd tht u remain sick


In [59]:
def user_in(review):
    tst_df = spark.createDataFrame([(review)],["statement"])
    token = Tokenizer(inputCol = "statement",outputCol="words")
    s_rem = StopWordsRemover(inputCol=token.getOutputCol(),outputCol="filtered")
    tf = HashingTF(inputCol=s_rem.getOutputCol(),outputCol = "features")
    tst_data = token.transform(tst_df)
    tst_data = s_rem.transform(tst_data)
    tst_data = tf.transform(tst_data)
    return tst_data

In [60]:
tst = user_in(review)

In [61]:
tst_pred=model.transform(tst)

In [62]:
tst_res=tst_pred.select("prediction",f.when(tst_pred["prediction"]==1.0,"Positive review").otherwise("Negative Review"))

In [63]:
tst_res=tst_res.withColumnRenamed("CASE WHEN (prediction = 1.0) THEN Positive review ELSE Negative Review END" ,"Review")

In [64]:
tst_res=tst_res.select("Review")

In [65]:
tst_res.show()

+---------------+
|         Review|
+---------------+
|Negative Review|
+---------------+

